In [1]:
%load_ext autoreload
%autoreload 2

In [13]:
import luigi
import pandas as pd
from factue.pipelines.clean_reference import CleanReferenceTask
from factue.utils.vars import PROJECT_ROOT
from factue.utils.viz import disp

import json

In [16]:
json_file = PROJECT_ROOT / "factue/methods/prompts/check_claim.json"
json_cont = json.load(open(json_file, "r"))

print(json_cont["layout"]["v002"])
print("*" * 50)
print(json_cont["task"]["v002"])
print("*" * 50)
print(json.dumps(json_cont["output"]["v001"]))

CLAIM:
{claim}

POST:
{text}
**************************************************
You are an expert data normalizer.
Your task is to receive raw, informal text and output a formal, concise version that captures only the essential facts in a single sentence.
- Be neutral and objective
- Avoid subjective opinions
- Only keep  information that has been found in original text
- Remove any unnecessary details or embellishments
- Do not include any personal opinions or interpretations
- Do not add any new information that is not present in the original claim
- Do not change the meaning of the original claim
- Do not include any disclaimers or hedging language
- Do not include any references to the source of the claim
- Do not include any additional context or background information
- Do not include any qualifiers or caveats
- Do not include any personal anecdotes or examples
-Do to verify the claim
**************************************************
{"questions": [{"id": "meaning_preserved", "a

In [4]:
sample_in = (
    PROJECT_ROOT / "data/01_preprocessed" / "dev" / "dev-eng" / "batch_0000.parquet"
)
df = pd.read_parquet(sample_in)
df

,text,reference,text_lang,text_lang_score,reference_lang,reference_lang_score
0,"The Karnofsky Jewish family, who immigrated to the United States from Lithuania, employed a 7-year-old boy and adopted (so to speak) him into their home. He was originally given homework to get f...",Photo shows Louis Armstrong as a child,en,1.00,en,0.95
1,Trophy hunting is horrific. 😥😥😥 Trophy hunting is horrific. 😥😥😥 Trophy hunting is horrific. 😥😥😥 mill\nCo\nThis little guy was rescued after his\nmother was murdered by a hunter.\nHis eyes say it a...,This leopard cub's mother was killed by a trophy hunter,en,0.94,en,0.99
2,Magarmacch // Heavy Rain //Hyderabad //Crocodile // Alert Magarmacch // Heavy Rain //Hyderabad //Crocodile // Alert Magarmacch // Heavy Rain //Hyderabad //Crocodile // Alert None,Videos show current situation of Hyderabad amid heavy rain,uk,0.10,en,0.66
3,"The salary of a U.S. Senator is $174,000 per year. This is Joe Biden’s house.... seems legit 🙄 The salary of a U.S. Senator is $174,000 per year. This is Joe Biden’s house.... seems legit 🙄 The sa...",Joe Biden lives in a large estate bought on a senator's salary,en,0.98,en,0.97
4,BREAKING NEWS | HUGE Explosion and chaos outside Kabul airport. Air strikes in Afghanistan are confirmed but whether that is US military or otherwise is not known. Update. 40 dead. 120 injured acc...,"Photo shows August 26, 2021 explosion near Kabul airport",en,0.96,en,0.97
5,Né? Né? Né? So in total white South\nAfricans own 357507 ha of\nurban land and 26663144 ha\nof agricultural land (which\nincludes vast areas of semi-\ndesert Karoo) making a total\nof 27020651 ha....,White people own only 22 percent of South Africa’s land,en,0.82,en,0.84
6,Amitabh Bachchan Tested Positive for Covid-19. He Shared this video from the Hospital !! Amitabh Bachchan Tested Positive for Covid-19. He Shared this video from the Hospital !! Amitabh Bachchan T...,This video shows Amitabh Bachchan thanking healthcare workers after he was hospitalised for COVID-19 in July 2020,en,0.78,en,0.96
7,Thank you for letting us know Ruth Swissa! Well done Publix! 💙🇮🇱💙 Thank you for letting us know Ruth Swissa! Well done Publix! 💙🇮🇱💙 Thank you for letting us know Ruth Swissa! Well done Publix! 💙🇮🇱...,Publix supermarket chain will stop carrying Ben & Jerry's,en,0.55,en,0.79
8,I'm old enough to remember when hydroxychloroquine was a kook theory from a kook president. I'm old enough to remember when hydroxychloroquine was a kook theory from a kook president. I'm old enou...,American Medical Association Rescinds Previous Statement Against Prescription of Hydroxychloroquine to COVID-19 Patients.,en,0.95,en,0.70
9,ब्राज़ील के टेलीविज़न चैनलों ने भारत के प्रधानमंत्री मोदी का भाषण अपने देश में दिखाया और कल रात को ही ब्राज़ीलिओ ने ये कर दिखाया। ब्राज़ील के टेलीविज़न चैनलों ने भारत के प्रधानमंत्री मोदी का भाषण अपने ...,"Brazilains participating in India’s nationwide COVID-19 candle light vigil, which Indian Prime Minister Narendra Modi called for on April 3, 2020.",hi,1.00,en,0.81


In [9]:
t1 = CleanReferenceTask(
    input_path=sample_in,
    input_dir=PROJECT_ROOT / "data/01_preprocessed",
    output_dir=PROJECT_ROOT / "data/02_cleaned_reference_azure_openai",
    force=True,
)

luigi.build([t1], local_scheduler=True)

/Users/marcinsawinski/Documents/GitHub/factue-task2/.venv/lib/python3.12/site-packages/luigi/parameter.py:296: UserWarning: Parameter "input_path" with value "/Users/marcinsawinski/Documents/GitHub/factue-task2/data/01_preprocessed/dev/dev-eng/batch_0000.parquet" is not of type string.
  warnings.warn('Parameter "{}" with value "{}" is not of type string.'.format(param_name, param_value))
/Users/marcinsawinski/Documents/GitHub/factue-task2/.venv/lib/python3.12/site-packages/luigi/parameter.py:296: UserWarning: Parameter "input_dir" with value "/Users/marcinsawinski/Documents/GitHub/factue-task2/data/01_preprocessed" is not of type string.
  warnings.warn('Parameter "{}" with value "{}" is not of type string.'.format(param_name, param_value))
/Users/marcinsawinski/Documents/GitHub/factue-task2/.venv/lib/python3.12/site-packages/luigi/parameter.py:296: UserWarning: Parameter "output_dir" with value "/Users/marcinsawinski/Documents/GitHub/factue-task2/data/02_cleaned_reference_azure_opena

Converted /Users/marcinsawinski/Documents/GitHub/factue-task2/data/01_preprocessed/dev/dev-eng/batch_0000.parquet -> /Users/marcinsawinski/Documents/GitHub/factue-task2/data/02_cleaned_reference_azure_openai/dev/dev-eng/batch_0000.parquet


True

In [8]:
sample_out = (
    PROJECT_ROOT
    / "data/02_cleaned_reference_test"
    / "dev"
    / "dev-eng"
    / "batch_0000.parquet"
)
df = pd.read_parquet(sample_out)
disp(df[["text", "reference", "check"]])

,text,reference,check
0,"The Karnofsky Jewish family, who immigrated to the United States from Lithuania, employed a 7-year-old boy and adopted (so to speak) him into their home. He was originally given homework to get food because he was a starving kid. He remained under the Jewish families employ, until he was 12 Karnofsky gave him money to buy his first instrument, which was a common instrument in Jewish families. They really admired his musical talent.Later, when he became a professional … See More The Karnofsky Jewish family, who immigrated to the United States from Lithuania, employed a 7-year-old boy and adopted (so to speak) him into their home. He was originally given homework to get food because he was a starving kid. He remained under the Jewish families employ, until he was 12 Karnofsky gave him money to buy his first instrument, which was a common instrument in Jewish families. They really admired his musical talent.Later, when he became a professional … See More The Karnofsky Jewish family, who immigrated to the United States from Lithuania, employed a 7-year-old boy and adopted (so to speak) him into their home. He was originally given homework to get food because he was a starving kid. He remained under the Jewish families employ, until he was 12 Karnofsky gave him money to buy his first instrument, which was a common instrument in Jewish families. They really admired his musical talent.Later, when he became a professional … See More None",Photo shows Louis Armstrong as a child,"{ 'questions': [ {'id': 'meaning_preserved', 'answer': 'false', 'errors': ['The claim does not mention the Karnofsky family employing Louis Armstrong as a child, but rather that they employed a 7-year-old boy who later became a professional musician, which is likely referring to Louis Armstrong.']}, {'id': 'correct_named_entities', 'answer': 'true'}, {'id': 'correct_numbers', 'answer': 'false', 'errors': ['The claim mentions the child being shown as a photo, but does not mention any specific age or number related to the Karnofsky family employing him. However, it is mentioned that he was 7 years old when employed and 12 when no longer under their employ.']}, {'id': 'no_added_data', 'answer': 'false', 'errors': ['The claim mentions a photo of Louis Armstrong as a child, which is not present in the original text.']}, {'id': 'no_missing_data', 'answer': 'true'}, {'id': 'alternative_claim_formulation', 'answer': 'true', 'errors': ['A more accurate formulation would be: ""The Karnofsky Jewish family employed a 7-year-old boy who later became Louis Armstrong.""']} ] }"
1,"Trophy hunting is horrific. 😥😥😥 Trophy hunting is horrific. 😥😥😥 Trophy hunting is horrific. 😥😥😥 mill Co This little guy was rescued after his mother was murdered by a hunter. His eyes say it all, don't you think? If you want to ban trophy hunting forever, spread this everywhere. Image Credit: FB. Changes: cropped, resized, text added. https://bit.ly/3b2vwas",This leopard cub's mother was killed by a trophy hunter,"{ 'questions': [ {'id': 'meaning_preserved', 'answer': 'true'}, {'id': 'correct_named_entities', 'answer': 'false', 'errors': ['""leopard cub"" is not present in the original text']}, {'id': 'correct_numbers', 'answer': 'true'}, {'id': 'no_added_data', 'answer': 'false', 'errors': ['""rescued"", ""murdered"", ""Image Credit: FB. Changes: cropped, resized, text added."" are not present in the original text']}, {'id': 'no_missing_data', 'answer': 'true'}, {'id': 'alternative_claim_formulation', 'answer': 'true'} ] }"
2,Magarmacch // Heavy Rain //Hyderabad //Crocodile // Alert Magarmacch // Heavy Rain //Hyderabad //Crocodile // Alert Magarmacch // Heavy Rain //Hyderabad //Crocodile // Alert None,Videos show current situation of Hyderabad amid heavy rain,"{ 'questions': [ {'id': 'meaning_preserved', 'answer': 'false', 'errors': ['The claim does not mention the heavy rain or crocodile alert, which are present in the original text.']}, {'id': 'correct_named_entities', 'a